### 엘리스 환경에서의 Pyspark 환경 구성
---
엘리스에서 Pyspark 실습을 위해 환경을 구성합니다. <br>
실습 코드 실행 전 **아래의 셀을 꼭 실행**해주세요.

**실습에서는 2.4.7 버전으로 진행하나** 엘리스 환경에서의 호환을 위해 **최신 버전을 설치**합니다.

In [2]:
!pip install pyspark
!pip install Koalas

import databricks.koalas as ks
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Pyspark 정상 동작 확인

In [3]:
spark

### Data read
---
데이터를 불러오고 컬럼명을 조정합니다.

In [4]:
df = spark.read.format("csv").load("./delivery_data.csv")

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([\
    StructField("날짜", StringType(), True),\
    StructField("시간대별시간", IntegerType(), True),\
    StructField("배달상점광역시도명", StringType(), True),\
    StructField("배달상점시군구명", StringType(), True),\
    StructField("배달상점법정동명", StringType(), True),\
    StructField("주문건수", IntegerType(), True)])

df = spark.read.options(delimiter=',').option("header", True).schema(schema).csv("./delivery_data.csv")

적용된 컬럼을 확인합니다.

In [6]:
df.printSchema()

root
 |-- 날짜: string (nullable = true)
 |-- 시간대별시간: integer (nullable = true)
 |-- 배달상점광역시도명: string (nullable = true)
 |-- 배달상점시군구명: string (nullable = true)
 |-- 배달상점법정동명: string (nullable = true)
 |-- 주문건수: integer (nullable = true)



### Koalas

In [7]:
ks.set_option('compute.ops_on_diff_frames', True)

In [8]:
kdf = df.to_koalas()

In [9]:
kdf.head(10)

,날짜,시간대별시간,배달상점광역시도명,배달상점시군구명,배달상점법정동명,주문건수
0,2019-08-01,0,경기도,고양시 일산동구,마두동,4
1,2019-08-01,0,경기도,고양시 일산동구,백석동,28
2,2019-08-01,0,경기도,고양시 일산동구,식사동,3
3,2019-08-01,0,경기도,고양시 일산동구,장항동,4
4,2019-08-01,0,경기도,고양시 일산동구,정발산동,2
5,2019-08-01,0,경기도,고양시 일산서구,일산동,7
6,2019-08-01,0,경기도,고양시 일산서구,탄현동,3
7,2019-08-01,0,경기도,광명시,광명동,9
8,2019-08-01,0,경기도,광명시,철산동,13
9,2019-08-01,0,경기도,광명시,하안동,7


In [10]:
kdf.shape

(426971, 6)

In [11]:
tst = kdf.groupby("배달상점광역시도명").sum().sort_values(by="주문건수", ascending=False).head(5)

In [12]:
tst.to_parquet("koalas_res")

In [13]:
kdf

,날짜,시간대별시간,배달상점광역시도명,배달상점시군구명,배달상점법정동명,주문건수
0,2019-08-01,0,경기도,고양시 일산동구,마두동,4
1,2019-08-01,0,경기도,고양시 일산동구,백석동,28
2,2019-08-01,0,경기도,고양시 일산동구,식사동,3
3,2019-08-01,0,경기도,고양시 일산동구,장항동,4
4,2019-08-01,0,경기도,고양시 일산동구,정발산동,2
5,2019-08-01,0,경기도,고양시 일산서구,일산동,7
6,2019-08-01,0,경기도,고양시 일산서구,탄현동,3
7,2019-08-01,0,경기도,광명시,광명동,9
8,2019-08-01,0,경기도,광명시,철산동,13
9,2019-08-01,0,경기도,광명시,하안동,7


In [14]:
a=kdf.groupby("시간대별시간").count()

In [15]:
a

,날짜,배달상점광역시도명,배달상점시군구명,배달상점법정동명,주문건수
시간대별시간,,,,,
12,28864,28864,28864,28864,28864
22,25349,25349,25349,25349,25349
1,7880,7880,7880,7880,7880
13,28373,28373,28373,28373,28373
6,671,671,671,671,671
16,29433,29433,29433,29433,29433
3,1386,1386,1386,1386,1386
20,33532,33532,33532,33532,33532
5,761,761,761,761,761


In [16]:
a=kdf.groupby("시간대별시간").count().iloc[:,-1:]

In [17]:
b=kdf.groupby("시간대별시간").mean()

In [18]:
c=kdf.groupby("시간대별시간").std()

In [19]:
b.rename(columns = {'주문건수' : 'mean'}, inplace=True)

In [20]:
c.rename(columns = {'주문건수' : 'std'}, inplace=True)

In [21]:
bc = ks.concat([b,c],axis=1)

In [22]:
a.rename(columns = {'주문건수' : 'count'}, inplace=True)

In [23]:
abc = ks.concat([a,bc],axis=1)

In [24]:
abc = abc.sort_index()

In [25]:
abc

,count,mean,std
시간대별시간,,,
0,12669,5.124872,6.199660
1,7880,4.248223,4.382471
2,3446,3.941381,4.137062
3,1386,4.029582,3.695229
4,1035,3.917874,3.441978
5,761,3.617608,3.272236
6,671,3.780924,3.014590
7,697,3.984218,3.186454
8,957,4.051202,3.554197


In [26]:
abc.to_parquet('koalas_describe')

---
본 페이지를 통해 제공하는 모든 자료는 저작권법에 의해 보호받는 ㈜엘리스의 자산이며, 무단 사용 및 도용, 복제 및 배포를 금합니다.

해당 실습에 활용된 데이터는 [KT 통신 빅데이터 플랫폼, 시간-지역별 배달 주문건수](https://www.bigdata-telecom.kr/invoke/SOKBP2603/?goodsCode=KGUTIMEORDER)의 자료를 활용하였습니다.

Copyright 2021 엘리스 Inc. All rights reserved.